In [1]:
import os
import utils
import warnings

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xgboost as xgb

from bayes_opt import BayesianOptimization
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import mean_squared_error as mse
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import train_test_split
from tqdm import tqdm_notebook as tqdm
from xgboost import XGBRegressor

from constants import DATA_DIR
from constants import PUNT_TYPES
warnings.filterwarnings('ignore')

In [2]:
cv = RepeatedKFold(n_splits=2, n_repeats=100)

### Ridge Regression

In [13]:
def ridge(X, y):
    parameters = {'alpha':[0.1, 1, 10]}
    Ridge_reg= GridSearchCV(Ridge(), parameters, scoring='neg_root_mean_squared_error', cv=cv, n_jobs=-1)
    Ridge_reg.fit(X, y)
    best_ridge = Ridge_reg.best_estimator_
    errors = utils.cross_val(best_ridge, X, y, n_folds=3)
    print (Ridge_reg.best_params_)
    return Ridge_reg.best_params_, np.mean(errors['MAE']['valid']), np.mean(errors['RMSE']['valid'])

### Lasso Regression

In [4]:
def lasso(X, y):
    parameters = {'alpha':[1e-4, 1e-3, 1e-2, 1e-1]}
    lasso_result = GridSearchCV(Lasso(), parameters, scoring='neg_root_mean_squared_error',cv=cv,n_jobs=-1)
    lasso_result.fit(X, y)
    best_model = lasso_result.best_estimator_
    errors = utils.cross_val(best_model, X, y, n_folds=3)
    print(lasso_result.best_params_)
    return lasso_result.best_params_, np.mean(errors['MAE']['valid']), np.mean(errors['RMSE']['valid'])

### Vanilla XGBRegressor

In [5]:
def vanilla_xgb(X,y):
    vanilla_reg = XGBRegressor().fit(X, y)
    errors = utils.cross_val(vanilla_reg, X, y, n_folds=3)
    utils.summarize_errors(errors, verbose=0)
    return None, np.mean(errors['MAE']['valid']), np.mean(errors['RMSE']['valid'])

###  XGBRegressor w/Bayesian Optimized Hyperparameters

In [11]:
def bo_tune_xgb(max_depth, gamma, n_estimators, learning_rate, subsample):
    params = {'gamma': gamma, 
              'learning_rate':learning_rate, 
              'max_depth': int(max_depth), 
              'n_estimators': int(n_estimators), 
              'subsample': subsample, 
              'eta': 0.1, 
              'eval_metric': 'rmse'}
    
    #scores = cross_val_score(XGBRegressor(random_state=1, **params), X, y, scoring='neg_root_mean_squared_error', cv=3).mean()
    #score = scores.mean()
    #return score
    dtrain = xgb.DMatrix(X, y)
    cv_result = xgb.cv(params, dtrain, num_boost_round=70, nfold=3)
    #Return the negative RMSE
    return -1.0 * cv_result['test-rmse-mean'].iloc[-1]
    

In [7]:
def bo_xgb(X, y):
    params = {'max_depth': (3, 10), 
              'gamma': (0, 1), 
              'n_estimators':(100,120), 
              'learning_rate':(0,1), 
              'subsample': (0.8, 1)}
    xgb_bo = BayesianOptimization(bo_tune_xgb, params, random_state = 1)
    xgb_bo.maximize(n_iter=5, init_points=20, acq='ei')
    params = xgb_bo.max['params']
    params['max_depth']= int(params['max_depth'])
    params['n_estimators']= int(params['n_estimators'])
    print(params)
    clf = XGBRegressor(**params)
    errors = utils.cross_val(clf, X, y, n_folds=3)
    return params, np.mean(errors['MAE']['valid']), np.mean(errors['RMSE']['valid'])

### Neural Network

In [8]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold
from sklearn import preprocessing
from keras import regularizers
from keras.callbacks import ModelCheckpoint
import math

In [9]:
def get_modelcheckpoint_path():
    #Create a file path for a model and save models in hdf5 files with datetime, validation losses and epochs
    parent = DATA_DIR+"/NN/"
    child = pd.Timestamp.now().strftime('%Y%m%d-%Hh%Mm') + "-model-epoch_{epoch:02d}-rmse_{val_loss:.5f}.hdf5"
    return parent + child 

def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(X.shape[1], input_dim=X.shape[1], kernel_initializer='normal', activation='relu'))
    model.add(Dense(15, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal'))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

def dropout_model():
    # create model
    model = Sequential()
    model.add(Dense(X.shape[1], input_dim=X.shape[1], kernel_initializer='normal', activation='relu'))
    model.add(Dense(15, kernel_initializer='normal', activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(1, kernel_initializer='normal'))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

def create_regressor(model, verbose=0):
    reg = KerasRegressor(build_fn=model, epochs=100, batch_size=5, validation_split=0.2, shuffle=True, verbose=verbose)
    return reg

def plot_learning_process(history):
    # summarize history for loss
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
    return None

def base_keras_model(X, y):
    base_keras = create_regressor(baseline_model, 0)
    errors = utils.cross_val(base_keras, X, y, n_folds=5)
    return None, np.mean(errors['MAE']['valid']), np.mean(errors['RMSE']['valid'])

def dropout_keras_model(X, y):
    dropout_keras = create_regressor(dropout_model, 0)
    errors = utils.cross_val(dropout_keras, X, y, n_folds=5)
    return None, np.mean(errors['MAE']['valid']), np.mean(errors['RMSE']['valid'])

Based on all models above, lasso regression seemed to yield the lowest RMSE and MAE using kfold validation to evaluate errors.

In [14]:
rmse_table = pd.DataFrame()
mae_table = pd.DataFrame()
params_table = pd.DataFrame()
models = [ridge, lasso, vanilla_xgb, bo_xgb]#, base_keras_model, dropout_keras_model]
for punt in tqdm(PUNT_TYPES):
    if punt == []:
        punt = 'Base'
    else:
        punt = '+'.join(punt)
    df = pd.read_csv(os.path.join(DATA_DIR,'ABT', punt+'.csv'))
    X = df.loc[:, df.columns != 'VALUE']
    y = df['VALUE'].values.reshape(-1,1).flatten()
    for model in tqdm(models):
        param, mae_value, rmse_value = model(X, y)
        #print(param)
        #params_table.loc [punt, model.__name__] = param
        mae_table.loc[punt, model.__name__] = mae_value
        rmse_table.loc[punt, model.__name__] = rmse_value


{'alpha': 1}



{'alpha': 0.001}




   <--- Validation Errors --->
MAE  | Mean: 1.19116, SD: 0.0259
RMSE | Mean: 1.66879, SD: 0.02855

|   iter    |  target   |   gamma   | learni... | max_depth | n_esti... | subsample |
-------------------------------------------------------------------------------------
[16:03:34] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/learner.cc:541: 
Parameters: { n_estimators } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[16:03:34] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/learner.cc:541: 
Parameters: { n_estimators } might not be used.

  This may not be accurate due to some parameters are only used in language bindings

|  6        | -1.5      |  0.8946   |  0.08504  |  3.273    |  103.4    |  0.9756   |
[16:03:39] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/learner.cc:541: 
Parameters: { n_estimators } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[16:03:39] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/learner.cc:541: 
Parameters: { n_estimators } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[16:03:39] WARNING: /opt/concour

|  12       | -1.712    |  0.05336  |  0.5741   |  4.027    |  111.8    |  0.94     |
[16:03:46] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/learner.cc:541: 
Parameters: { n_estimators } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[16:03:46] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/learner.cc:541: 
Parameters: { n_estimators } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[16:03:46] WARNING: /opt/concour

...



   <--- Validation Errors --->
MAE  | Mean: 0.85659, SD: 0.01988
RMSE | Mean: 1.21758, SD: 0.02614

|   iter    |  target   |   gamma   | learni... | max_depth | n_esti... | subsample |
-------------------------------------------------------------------------------------
[16:11:38] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/learner.cc:541: 
Parameters: { n_estimators } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[16:11:38] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/learner.cc:541: 
Parameters: { n_estimators } might not be used.

  This may not be accurate due to some parameters are only used in language binding

|  6        | -1.104    |  0.8946   |  0.08504  |  3.273    |  103.4    |  0.9756   |
[16:11:43] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/learner.cc:541: 
Parameters: { n_estimators } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[16:11:43] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/learner.cc:541: 
Parameters: { n_estimators } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[16:11:43] WARNING: /opt/concour

|  12       | -1.285    |  0.05336  |  0.5741   |  4.027    |  111.8    |  0.94     |
[16:11:49] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/learner.cc:541: 
Parameters: { n_estimators } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[16:11:49] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/learner.cc:541: 
Parameters: { n_estimators } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[16:11:49] WARNING: /opt/concour

|  18       | -1.268    |  0.8959   |  0.4281   |  9.754    |  113.3    |  0.9243   |
[16:11:56] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/learner.cc:541: 
Parameters: { n_estimators } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[16:11:56] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/learner.cc:541: 
Parameters: { n_estimators } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[16:11:56] WARNING: /opt/concour

|  24       | -2.94     |  0.98     |  0.0      |  3.696    |  116.2    |  0.8843   |
[16:12:03] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/learner.cc:541: 
Parameters: { n_estimators } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[16:12:03] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/learner.cc:541: 
Parameters: { n_estimators } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[16:12:03] WARNING: /opt/concour

In [15]:
print(rmse_table)
print(mae_table)

            ridge     lasso  vanilla_xgb    bo_xgb
Base     1.320886  1.320647     1.668789  1.479503
FG%      1.153468  1.154564     1.482602  1.295209
FT%      1.134134  1.130243     1.427735  1.269910
PTS      1.095087  1.091077     1.393531  1.245949
TRB      1.118707  1.114890     1.422590  1.248522
AST      1.176265  1.172537     1.485554  1.320890
STL      1.116850  1.112735     1.383463  1.227914
BLK      1.162296  1.157925     1.455579  1.276767
FG%+TRB  0.989587  0.990648     1.258829  1.112071
BLK+FG%  1.032651  1.033386     1.291950  1.163333
AST+STL  1.026803  1.023345     1.285576  1.141569
PTS+FT%  0.944548  0.940592     1.217578  1.071859
            ridge     lasso  vanilla_xgb    bo_xgb
Base     0.886549  0.883271     1.191164  1.040129
FG%      0.765896  0.764749     1.042103  0.896351
FT%      0.766997  0.762777     1.026758  0.904212
PTS      0.692670  0.689151     0.942328  0.846986
TRB      0.725531  0.721972     0.994250  0.858593
AST      0.757410  0.753410    

In [16]:
min_mae = mae_table.idxmin(axis = 1)
print(min_mae)
min_rmse = rmse_table.idxmin(axis = 1)
print(min_rmse)

Base       lasso
FG%        lasso
FT%        lasso
PTS        lasso
TRB        lasso
AST        lasso
STL        lasso
BLK        lasso
FG%+TRB    lasso
BLK+FG%    lasso
AST+STL    lasso
PTS+FT%    lasso
dtype: object
Base       lasso
FG%        ridge
FT%        lasso
PTS        lasso
TRB        lasso
AST        lasso
STL        lasso
BLK        lasso
FG%+TRB    ridge
BLK+FG%    ridge
AST+STL    lasso
PTS+FT%    lasso
dtype: object
